#### InnoBioDiv


# Farmbot Watering System Demo

This notebook will walk you through the basic steps of the watering system of the Farmbot. Since it can actually be used to handle the farmbot, it is not possible to execute the code from your personal laptop.

In [1]:
from demo_functions import demo, Peripherals_WaterValve
d = demo()

ModuleNotFoundError: No module named 'demo_functions'

In [2]:
import pandas as pd

## Configure experiment
### 1. Edit plant configuration

The plant configuration is saved in a csv file in the same dictonary as this notebook. It contains the configuration for two plants named Demo1 and Demo2.
* x_position y_position and z_position: specify the position of the plants on the Farmbot field relative to the origin. 
* Moisture sensor: specifies which moisture sensors should be used.
* Desired soil moisture value: Value below which the plant is watered.(Higher values -> soil is dryer)
* temperature sensors: not used in this example
* action: specifies the action, in this case we want to water the plant depending on the soil moisture.
* watering duration in ms: how long the plant should be watered, corresponds to a specific amount of water.
* frequency of watering (not used with the option "water depending on soil moisture")

 <img  src="farmbot.jpg" width = "700" >
 


#### <div class="alert alert-success"> **Task:** choose new x and y values for the position of the two plants and change the values of the csv-file accordingly. Later in the Lab you can test if you can find the right positions on the FarmBot field. </div>

In [13]:
config = pd.read_csv('config.csv')

In [14]:
config

,id,plant_name,x_position,y_position,z_position,moisture_sensor_1,moisture_sensor_2,moisture_sensor_3,moisture_sensor_4,desired_soil_moisture_value,temperature_sensor_top_nr,temperature_sensor_bottom_nr,desired_soil_temperature_value,action,watering_duration_in_ms,last_time_watering,frequence_watering
0,1,Demo1,1000,1000,50,0,0,1,1,1500,0,1,0,watering depending on soil moisture,300,0,4000
1,2,Demo2,500,500,50,0,0,1,1,1500,0,1,0,watering depending on soil moisture,300,0,4000



### 2. Import plant config into database

Now you have to upload the changed csv-file to the database:

In [23]:
d.export_csv('config.csv')

### 2.1 Verify that the data has been transfered

In [24]:
print(d.fetch_plant_conf())

[(1, 'Demo1', 1000.0, 1000.0, 50.0, 0, 0, 1, 1, 2500.0, 0, 1, 0.0, 'watering depending on soil moisture', 300, '0', 4000.0), (2, 'Demo2', 500.0, 500.0, 50.0, 0, 0, 1, 1, 1500.0, 0, 1, 0.0, 'watering depending on soil moisture', 300, '0', 4000.0)]


## 3. Watering Script

> FCS = Farmbot Control System

1. flush watering nozzle
2. compare sensor values to the desired values in the configuration
3. water plants accordingly to the plan

### 3.1 Flush watering nozzle
To get rid of air bubbles in the pump which compromize the accuracy of the watering process, the water nozzle needs to be flushed before watering the plants:

In [9]:
d.bot.move_all_axis(*d.tube_clean_pos)
d.bot.enable_peripheral_permanent(Peripherals_WaterValve())
d.bot.wait_time(d.tube_clean_duration)
d.bot.disable_peripheral_permanent(Peripherals_WaterValve())
d.bot.wait_time(int(2**(((d.tube_clean_duration+100)/300)+7)+1000))  


[FCS]21-Nov-22 15:46:24 --> INFO : Move to x: 50 y: 50 z: -150 at the same time.
[FCS]21-Nov-22 15:46:26 --> INFO : Enable peripheral Water Valve permanently.
[FCS]21-Nov-22 15:46:27 --> INFO : wait for 1000 milliseconds.
[FCS]21-Nov-22 15:46:29 --> INFO : Disable peripheral Water Valve permanently.
[FCS]21-Nov-22 15:46:30 --> INFO : wait for 2625 milliseconds.


### 3.2 Fetch configuration from database

In [25]:
conf = d.fetch_plant_conf()
print(conf)
print(conf[0][2:5])
print(conf[0][14])

[(1, 'Demo1', 1000.0, 1000.0, 50.0, 0, 0, 1, 1, 2500.0, 0, 1, 0.0, 'watering depending on soil moisture', 300, '0', 4000.0), (2, 'Demo2', 500.0, 500.0, 50.0, 0, 0, 1, 1, 1500.0, 0, 1, 0.0, 'watering depending on soil moisture', 300, '0', 4000.0)]
(1000.0, 1000.0, 50.0)
300


### 3.3 Fetch sensor values from the data base for plant 1 and water accordingly

Get the values of the moisture sensors from the data base. If there is more than one sensor involved the average of the measurement values is used. In this example there are no real sensors but example values are provided from the database.

In [26]:
vals = d.fetch_values(conf[0])
mean = d.val_mean(vals)
print(mean)

if mean > conf[0][9]:
    x,y,z = conf[0][2:5]
    d.water_plant(x,y,z,conf[0][14],dry_run=False)

2200.85


### 3.4 Water plant 2

In [21]:
vals = d.fetch_values(conf[1])
mean = d.val_mean(vals)
print(mean)

if mean > conf[1][9]:
    x,y,z = conf[1][2:5]
    d.water_plant(x,y,z,conf[1][14],dry_run=False)

[FCS]21-Nov-22 15:56:17 --> INFO : Move to x: 500.0 y: 500.0 z: 50.0 at the same time.


2200.85


[FCS]21-Nov-22 15:56:36 --> INFO : Enable peripheral Water Valve permanently.
[FCS]21-Nov-22 15:56:37 --> INFO : wait for 300 milliseconds.
[FCS]21-Nov-22 15:56:37 --> INFO : Disable peripheral Water Valve permanently.
[FCS]21-Nov-22 15:56:38 --> INFO : wait for 1322 milliseconds.


### <img  src="soil_moisture.jpg" width = "900" >


#### <div class="alert alert-success"> **Task during the lab**: Check the mean value from the moisture sensors. What can you say about the soil moisture using the figure above? </div>

#### <div class="alert alert-success"> **Task:** Change the desired moisture value in the configuration csv so that the plants are no longer watered because the soil is already wet enough. </div>